In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
%matplotlib inline
sns.set(style="whitegrid")
#sns.set(rc={'figure.figsize':(10,10)})
import matplotlib
#matplotlib.use('agg')
#matplotlib.style.use('ggplot')
import pickle as pkl
from matplotlib import pyplot as plt
from scipy.stats.stats import pearsonr
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en import LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES
lemmatizer = Lemmatizer(LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES)
from sklearn.model_selection import cross_val_score,GridSearchCV
from sklearn.linear_model import LinearRegression,Ridge,Lasso

In [2]:
def lemma_maker(x, y):
    #print(lemmatizer(x,y)[0])
    return lemmatizer(x,y)[0]

In [3]:
br_to_us=pd.read_excel("Book.xlsx",skiprows=[0])
br_to_us_dict=dict(zip(br_to_us.UK.tolist(),br_to_us.US.tolist()))
spelling_replacement={'modifier':br_to_us_dict,'head':br_to_us_dict}

In [6]:
features_decades = pd.read_pickle("/data/dharp/compounding/datasets/Dist_Features_Temporal_non_Contextual.pkl")
#features_decades.set_index(["modifier", "head"], inplace = True)
features_decades.info()
features_decades.head()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 767266 entries, (a_noun, a_noun) to (zzzz_noun, best_noun)
Columns: 126 entries, 1800_local_mi to 2000_sim_with_modifier
dtypes: float64(126)
memory usage: 749.3+ MB


1800_local_mi  1810_local_mi  1820_local_mi  \
modifier head                                                         
a_noun   a_noun              0.001672       0.003646       0.003031   
         aa_noun             0.001672       0.003646       0.003031   
         aaa_noun            0.001672       0.003646       0.003031   
         american_noun       0.001672       0.003646       0.003031   
         anti_noun           0.001672       0.003646       0.003031   

                        1830_local_mi  1840_local_mi  1850_local_mi  \
modifier head                                                         
a_noun   a_noun              0.002096       0.002751       0.002647   
         aa_noun             0.002096       0.002751       0.002647   
         aaa_noun            0.002096       0.002751       0.002647   
         american_noun       0.002096       0.002751       0.002647   
         anti_noun           0.002096       0.002751       0.002647   

                        1860_local_mi  1870_local_mi  1880_local_mi  \
modifier head                                                         
a_noun   a_noun              0.002238       0.003202       0.002289   
         aa_noun             0.002238       0.003202       0.002289   
         aaa_noun            0.002238       0.003202       0.002289   
         american_noun       0.002238       0.003202       0.002289   
         anti_noun           0.002238       0.003202       0.002289   

                        1890_local_mi  ...  1910_sim_with_modifier  \
modifier head                          ...                           
a_noun   a_noun              0.001153  ...                     0.0   
         aa_noun             0.001153  ...                     0.0   
         aaa_noun            0.001153  ...                     0.0   
         american_noun       0.001153  ...                     0.0   
         anti_noun           0.001153  ...                     0.0   

                        1920_sim_with_modifier  1930_sim_with_modifier  \
modifier head                                                            
a_noun   a_noun                       0.004109                     0.0   
         aa_noun                      0.000000                     0.0   
         aaa_noun                     0.000000                     0.0   
         american_noun                0.000000                     0.0   
         anti_noun                    0.000000                     0.0   

                        1940_sim_with_modifier  1950_sim_with_modifier  \
modifier head                                                            
a_noun   a_noun                            0.0                0.006807   
         aa_noun                           0.0                0.000913   
         aaa_noun                          0.0                0.000000   
         american_noun                     0.0                0.000000   
         anti_noun                         0.0                0.000000   

                        1960_sim_with_modifier  1970_sim_with_modifier  \
modifier head                                                            
a_noun   a_noun                       0.008311                0.009011   
         aa_noun                      0.000538                0.000890   
         aaa_noun                     0.004226                0.005014   
         american_noun                0.000000                0.000000   
         anti_noun                    0.000000                0.000000   

                        1980_sim_with_modifier  1990_sim_with_modifier  \
modifier head                                                            
a_noun   a_noun                       0.009811                0.007134   
         aa_noun                      0.000915                0.000562   
         aaa_noun                     0.004987                0.003966   
         american_noun                0.008823                0.009120   
         anti_noun                    

In [7]:
reddy11_study=pd.read_csv("/data/dharp/compounding/datasets/ijcnlp_compositionality_data/MeanAndDeviations.clean.txt",sep="\t")
#print(reddy11_study.columns)
reddy11_study.columns=['compound','to_divide']
reddy11_study['modifier_mean'],reddy11_study['modifier_std'],reddy11_study['head_mean'],reddy11_study['head_std'],reddy11_study['compound_mean'],reddy11_study['compound_std'],_=reddy11_study.to_divide.str.split(" ",7).str
reddy11_study['modifier'],reddy11_study['head']=reddy11_study['compound'].str.split(" ",2).str
reddy11_study.modifier=reddy11_study.modifier.str[:-2]
reddy11_study['head']=reddy11_study['head'].str[:-2]
reddy11_study.drop(['compound','to_divide'],axis=1,inplace=True)
reddy11_study['modifier']=np.vectorize(lemma_maker)(reddy11_study['modifier'],'noun')
reddy11_study['head']=np.vectorize(lemma_maker)(reddy11_study['head'],'noun')
reddy11_study.replace(spelling_replacement,inplace=True)
reddy11_study['modifier']=reddy11_study['modifier']+"_noun"
reddy11_study['head']=reddy11_study['head']+"_noun"
reddy11_study=reddy11_study.apply(pd.to_numeric, errors='ignore')
#reddy11_study.set_index(['modifier','head'],inplace=True)
reddy11_study.info()
reddy11_study.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 8 columns):
modifier_mean    90 non-null float64
modifier_std     90 non-null float64
head_mean        90 non-null float64
head_std         90 non-null float64
compound_mean    90 non-null float64
compound_std     90 non-null float64
modifier         90 non-null object
head             90 non-null object
dtypes: float64(6), object(2)
memory usage: 5.7+ KB


,modifier_mean,modifier_std,head_mean,head_std,compound_mean,compound_std,modifier,head
0,3.866667,1.117537,4.866667,0.339935,4.250000,0.871165,end_noun,user_noun
1,1.607143,1.654848,1.892857,1.496169,1.703704,1.717337,firing_noun,line_noun
2,2.821429,1.964935,4.862069,0.344828,3.827586,1.233693,game_noun,plan_noun
3,4.766667,0.422953,4.862069,0.344828,4.800000,0.476095,application_noun,form_noun
4,0.600000,0.800000,4.586207,1.099129,1.310345,1.020596,snail_noun,mail_noun


In [8]:
merge_df=reddy11_study.merge(features_decades.reset_index(),on=['modifier','head'],how='inner')
merge_df.set_index(["modifier", "head"], inplace = True)
merge_df.info()
merge_df.head()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 78 entries, (end_noun, user_noun) to (bank_noun, account_noun)
Columns: 132 entries, modifier_mean to 2000_sim_with_modifier
dtypes: float64(132)
memory usage: 81.9+ KB


,,modifier_mean,modifier_std,head_mean,head_std,compound_mean,compound_std,1800_local_mi,1810_local_mi,1820_local_mi,1830_local_mi,...,1910_sim_with_modifier,1920_sim_with_modifier,1930_sim_with_modifier,1940_sim_with_modifier,1950_sim_with_modifier,1960_sim_with_modifier,1970_sim_with_modifier,1980_sim_with_modifier,1990_sim_with_modifier,2000_sim_with_modifier
modifier,head,,,,,,,,,,,,,,,,,,,,,
end_noun,user_noun,3.866667,1.117537,4.866667,0.339935,4.250000,0.871165,0.001672,0.003646,0.003031,0.002096,...,0.000000,0.000000,0.000000,0.000000,0.530392,0.481366,0.494029,0.467544,0.427230,0.434449
firing_noun,line_noun,1.607143,1.654848,1.892857,1.496169,1.703704,1.717337,0.001672,0.003646,0.003031,0.002096,...,0.042148,0.423542,0.235692,0.199146,0.186670,0.155473,0.152375,0.118157,0.075856,0.113830
game_noun,plan_noun,2.821429,1.964935,4.862069,0.344828,3.827586,1.233693,0.001672,0.003646,0.003031,0.002096,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.493077,0.527281,0.494790,0.439049,0.431091
application_noun,form_noun,4.766667,0.422953,4.862069,0.344828,4.800000,0.476095,0.001672,0.003646,0.003031,0.002096,...,0.629569,0.612390,0.564383,0.611894,0.646288,0.604611,0.624531,0.635458,0.529771,0.548555
web_noun,site_noun,2.678571,1.691440,3.933333,1.181336,3.785714,1.205853,0.001672,0.003646,0.003031,0.002096,...,0.000000,0.005332,0.000000,0.000000,0.000000,0.154463,0.118817,0.227270,0.590880,0.335398


In [9]:
merge_df.to_csv('CompoundAgnostic_DecadeAware.csv',sep='\t')